In [2]:
from google.colab import drive

# Mount Google Drive
drive.mount('/content/drive')

Mounted at /content/drive


In [26]:
import pandas as pd
import math

In [3]:
target_sentences_df = pd.read_csv('/content/drive/MyDrive/PhD/Transcript - correct/target_sentences.csv')
target_sentences_df

,Text
0,još malo pa će izbori
1,naoblačilo se kao da će padati kiša
2,naša kola su stara 10 godina
3,otišao je u inostranstvo prošle godine i još s...
4,telefoni zvone po ceo dan i tebe traže
...,...
62,pojavio se u poslednjem trenutku
63,sanjao sam te pre neku noć
64,premijer je obećao viši standard
65,sutra je doček Nove Godine


In [4]:
pip install transformers

In [6]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch

model_name = 'gpt2'
# Load pre-trained GPT-2 model and tokenizer
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/548M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [7]:
def extract_words_and_probabilities(subwords, subword_probabilities):
    words = []
    word_probabilities = []

    current_word = ""
    current_probability = 1.0  # Initialize to 1.0 as we will multiply probabilities

    # Iterate through the subword probabilities
    for subword, probability in zip(subwords, subword_probabilities):
        # Check if the subword starts with the special token 'Ġ'
        if subword.startswith('Ġ'):
            # If we have a current word, add it to the list with its probability
            if current_word:
                words.append(current_word)
                word_probabilities.append(current_probability)

            # Reset current word and probability for the new word
            current_word = subword[1:]  # Remove 'Ġ' from the start
            current_probability = probability
        else:
            # Concatenate subwords to form the current word
            current_word += subword
            # Multiply probabilities for subwords within the same word
            current_probability *= probability

    # Add the last word and its probability
    if current_word:
        words.append(current_word)
        word_probabilities.append(current_probability)

    return words, word_probabilities

In [12]:
def calculate_word_probabilities(sentence, tokenizer = tokenizer, model = model):

    # Tokenize the input sentence
    input_ids = tokenizer.encode(sentence, return_tensors='pt')

    # Generate word probabilities using GPT-2 model
    with torch.no_grad():
        outputs = model(input_ids)
        logits = outputs.logits

    # Extract probabilities for each word
    word_probabilities = torch.softmax(logits, dim=-1).mean(dim=1)

    # Decode the tokens back to words
    decoded_words = tokenizer.convert_ids_to_tokens(input_ids[0].tolist())

    total_probability =  0
    subwords = []
    subwords_probabilities = []
    # Display word probabilities for each word
    for word, probability in zip(decoded_words, word_probabilities[0]):
      subwords.append(word)
      subwords_probabilities.append(probability)
      total_probability += probability

    words, probabilities = extract_words_and_probabilities(subwords, subwords_probabilities)

    return words, probabilities, total_probability

In [27]:
words_list = []
probabilities_list = []
target_sentence_list = []

for i in range(0,len(target_sentences_df)):
  sentence = target_sentences_df['Text'][i].lower()
  words = sentence.split(' ')
  _, probabilities, total  = calculate_word_probabilities(sentence)

  for word, prob in zip(words, probabilities):
    words_list.append(word)
    probabilities_list.append(-math.log2(prob.item()))
    target_sentence_list.append(i)

# Create a DataFrame
df = pd.DataFrame({'Sentence': target_sentence_list, 'Word': words_list, 'Surprisal GPT-2': probabilities_list})

# Display the DataFrame
print(df)

     Sentence    Word  Surprisal GPT-2
0           0     još        20.359610
1           0    malo        30.317091
2           0      pa        14.463784
3           0      će        33.935188
4           0  izbori        41.827941
..        ...     ...              ...
452        65   doček        41.954752
453        65    nove        20.024511
454        65  godine        20.886222
455        66   nadam        31.679341
456        66      se        13.018103

[457 rows x 3 columns]


In [30]:
# Save the DataFrame to a CSV file
csv_file_path = "/content/drive/MyDrive/PhD/Surprisal estimation/word_surprisals_gpt2.csv"
df.to_csv(csv_file_path, index=False)